In [1]:
'''
    (1) Adapted from template of IPAI course lab (inspiration to use to deepcopy and creation of Field class):
    
        https://github.com/hsu-ai-course/hsu.ai/blob/master/code/02.%20Ti%D1%81-Ta%D1%81-Toe.ipynb
    
    (2) Enums and their usage is inspired by the documentation:
    
        https://docs.python.org/3/library/enum.html
        
    (3) Usage of regular expressions is taken from official py doc:
    
        https://docs.python.org/3/library/re.html#re.match
'''

from enum import Enum, auto
from copy import deepcopy
from math import sqrt
from re import match

class Cell(Enum):
    EMPTY = auto()
    BLACK = auto()
    WHITE = auto()

class Player(Enum):
    HUMAN = auto()
    BOT   = auto()
    
class Field():
    def __init__(self, state = None):
        self.state = self.initial_state() if state is None else state
            
    def initial_state(self):
        '''
        The starting position of the game
        '''
        
        # initializing a structure to hold rows on cells
        temp_state = []
        
        # generating first row
        temp_state.append([Cell.BLACK if i % 2 else Cell.WHITE for i in range(4)])
        
        # two empty rows
        temp_state.append([Cell.EMPTY for _ in range(4)])
        temp_state.append([Cell.EMPTY for _ in range(4)])
        
        # adding last row which is first reversed
        temp_state.append(temp_state[0][::-1])
        
        return temp_state
    
    def __str__(self):
        '''
        Nice looking field represented by string
        '''
        
        def stringify_cell(cell):
            if cell == Cell.BLACK:
                return 'B'
            if cell == Cell.WHITE:
                return 'W'
            return ' '
        
        res = ''
        for i in range(4, 0, -1):
            res += f'{i} | ' + ' '.join(map(stringify_cell, self.state[i - 1])) + '\n'
        res += '- + - - - -\n'
        res += '  | A B C D\n'
        
        return res
    
    def get_player_moves(self, player):
        '''
        Returns the list of possible moves in format: ((x1, y1), (x2, y2))
        '''
        
        player_cell = Cell.WHITE if player == Player.HUMAN else Cell.BLACK
        movements = [(-1, 0), (1, 0), (0, -1), (0, 1)] # possible directions to move 
        
        res = []
        for i in range(4):
            for j in range(4):
                if self.state[i][j] == player_cell:
                    for move in movements:
                        move_start, move_end = (i, j), (i + move[0], j + move[1])
                        if self.move_possible(move_start, move_end, player_cell):
                            res.append((move_start, move_end))                   
        return res
                    
                
    def move_possible(self, start, end, player_cell):
        '''
        Checks if the start and end points are within the board boundaries
        And also checks if the move is legal
        '''
        
        if 0 <= start[0] <= 3 and 0 <= start[1] <= 3     and \
           0 <= end[0] <= 3   and 0 <= end[1] <= 3       and \
           self.state[start[0]][start[1]] == player_cell and \
           self.state[end[0]][end[1]]     == Cell.EMPTY  and \
           abs(start[0] - end[0]) + abs(start[1] - end[1]) == 1:
            return True
        return False
    
    def make_move(self, start, end):
        '''
        Changing the state of board by making a specified move
        '''
        
        self.state[start[0]][start[1]], self.state[end[0]][end[1]] = \
        self.state[end[0]][end[1]], self.state[start[0]][start[1]]
        
    def game_over(self):
        '''
        Checks if a player won, if yes - returns the player, if no - None
        '''
        
        # checking horizontal lines
        lines = [(0, 1, 2), (1, 2, 3)]
        for i in range(4):
            for a, b, c in lines:
                if self.state[i][a] == self.state[i][b] == self.state[i][c]:
                    if self.state[i][a] == Cell.WHITE:
                        return Player.HUMAN
                    elif self.state[i][a] == Cell.BLACK:
                        return Player.BOT
                    else:
                        continue
                
        # checking for vertical lines
        for j in range(4):
            for a, b, c, in lines:
                if self.state[a][j] == self.state[b][j] == self.state[c][j]:
                    if self.state[a][j] == Cell.WHITE:
                        return Player.HUMAN
                    elif self.state[a][j] == Cell.BLACK:
                        return Player.BOT
                    else:
                        continue
         
        # checking for diagonal lines
        diags = [((0, 0), (1, 1), (2, 2)),
                 ((0, 1), (1, 2), (2, 3)),
                 ((1, 0), (2, 1), (3, 2)),
                 ((1, 1), (2, 2), (3, 3)),
                 ((2, 0), (1, 1), (0, 2)),
                 ((2, 1), (1, 2), (0, 3)),
                 ((3, 0), (2, 1), (1, 2)),
                 ((3, 1), (2, 2), (1, 3))]
        for (x1, y1), (x2, y2), (x3, y3) in diags:
            if self.state[x1][y1] == self.state[x2][y2] == self.state[x3][y3]:
                if self.state[x1][y1] == Cell.WHITE:
                    return Player.HUMAN
                elif self.state[x1][y1] == Cell.BLACK:
                    return Player.BOT
                else:
                    continue
            
        # no winner
        return None
    
    def get_copy(self):
        '''
        Producing a full copy of current field
        '''
        
        return deepcopy(self)
    
    def estimate_position(self):
        '''
        Estimation of a current position on a board w.r.t. the player
        I selected to estimate the position based on the closeness of all cells of the player
        E.g. the closer to black cells to each other, the more pleasant position of for black
        
        The overall metric would be the difference in scores between white and black positions:
        If whites (humans) are better -> the score is positive
        If blacks (bots) are better -> the score is negative
        
        Analogy: chess
        
        In the context of minimax, the results are as follows:
            The human (white) would need to maximize the score (as it is positive)
            And bot (black) would need to minimize the score (to make it negative)
        '''
        def calc_score(field, player):
            # detecting all the cells of a player
            player_cell = Cell.WHITE if player == Player.HUMAN else Cell.BLACK
            cells = []
            for i in range(4):
                for j in range(4):
                    if field.state[i][j] == player_cell:
                        cells.append((i, j))
            
            # calculating the middle point of a cluster of cells
            mean = [0, 0]
            for x, y in cells:
                mean[0] += x
                mean[1] += y
            mean[0] /= 4
            mean[1] /= 4
            
            # calculating the distances of all points to this point
            s = 0
            for x, y in cells:
                s += sqrt((mean[0] - x) * (mean[0] - x) + (mean[1] - y) * (mean[1] - y))
                
            return 1 / s
        
        # getting the result of current game 
        game_res = self.game_over()
        
        # if the game is still going
        if not game_res:
            blacks = calc_score(self, Player.BOT)
            whites = calc_score(self, Player.HUMAN)
            return whites - blacks
            
        # if the game is over
        else:
            if game_res == Player.HUMAN:
                return 100
            else:
                return -100
    
class MinimaxTree():
    def __init__(self, field, turn, prev_move = None):
        self.field = field
        self.turn = turn
        self.children = []
        self.score = 0
        self.prev_move = prev_move
        
    def __repr__(self):
        '''
        A nice string representation of a minimax tree
        '''
        
        return f'Minimax tree with {len(self.children)} kids and turn of {self.turn}'
        
    def make_tree(self, depth):
        '''
        Making the tree with specified depth
        '''
        
        if depth:
            for move in self.field.get_player_moves(self.turn):
                temp_field = self.field.get_copy()
                temp_field.make_move(*move)
                self.children.append(MinimaxTree(temp_field,
                                                 Player.BOT if self.turn == Player.HUMAN else Player.HUMAN,
                                                 prev_move = move))
                
            for child in self.children:
                child.make_tree(depth - 1)
                
    def evaluate_tree(self):
        '''
        Using the minimax (maximization for human and minimization for bot)
        to recursively evaluate the scores on each tree node
        '''
        
        # position hasn't reached depth
        if self.children:
            
            # getting the score
            score = self.field.estimate_position()
            
            # if the game is over, truncating the tree and returning the score
            if score == 100 or score == -100:
                self.children = []
                self.score = score
                return score
            
            # if the game goes on, returning score according to min or max
            else:
                scores = []
                for child in self.children:
                    scores.append(child.evaluate_tree())
                    
                if self.turn == Player.HUMAN:
                    fin_score = max(scores)
                    self.score = fin_score
                    return fin_score
                else:
                    fin_score = min(scores)
                    self.score = fin_score
                    return fin_score
        
        # we reached a leaf node
        else:
            self.score = self.field.estimate_position()
            return self.field.estimate_position()
            
def minimax(field, depth = 5):
    '''
    A naive implementation of an AI for taktikl game
    '''
    
    # make tree with depth
    t = MinimaxTree(field, Player.BOT)
    t.make_tree(depth)
    
    # traverse the tree and analyze tree
    t.evaluate_tree()
    
    # for child in t.children:
    #     print(f"Move {child.prev_move} with score {child.score}")
    
    # return the best move according to minimax
    return min(t.children, key = lambda child: child.score).prev_move

def get_move_from_human(field):
    '''
    Handling the input from user
    '''
    
    inp = ''
    while True:
        inp = input('Your move (e.g. A2B2 or c1d1): ')
        translated = translate_move(inp)
        if bool(translated) and field.move_possible(*translated, Cell.WHITE):
            return translated
        else:
            print('Move is impossible or incorrect format of move')

def translate_move(raw_move):
    '''
    Translate the imput of user into coordinates which 
    represent potential moves
    '''
    
    raw_move = raw_move.lower()
    if bool(match('[a-d][1-4][a-d][1-4]', raw_move)):
        move  = raw_move
        start = ord(move[1]) - ord('1'), ord(move[0]) - ord('a')
        end   = ord(move[3]) - ord('1'), ord(move[2]) - ord('a')
        return (start, end)
    else:
        return None

In [2]:
# initializing the field
f = Field() 
turn = Player.HUMAN
print(f)

while True: # starting the game
    
    # getting the moves from players depending on turn of game
    move = None
    if turn == Player.BOT:
        move = minimax(f)
        print("Bot's move:")
    else:
        move = get_move_from_human(f)
        
    # players makes the move
    f.make_move(*move)
    print(f)
    
    # seeing if someone won
    res = f.game_over()
    if res:
        print(f'{res} won!!!')
        break
    
    turn = Player.HUMAN if turn == Player.BOT else Player.BOT

4 | B W B W
3 |        
2 |        
1 | W B W B
- + - - - -
  | A B C D



Your move (e.g. A2B2 or c1d1):  a1a2


4 | B W B W
3 |        
2 | W      
1 |   B W B
- + - - - -
  | A B C D

Bot's move:
4 | B W B W
3 |        
2 | W B    
1 |     W B
- + - - - -
  | A B C D



Your move (e.g. A2B2 or c1d1):  c1c2


4 | B W B W
3 |        
2 | W B W  
1 |       B
- + - - - -
  | A B C D

Bot's move:
4 |   W B W
3 | B      
2 | W B W  
1 |       B
- + - - - -
  | A B C D



Your move (e.g. A2B2 or c1d1):  d4d3


4 |   W B  
3 | B     W
2 | W B W  
1 |       B
- + - - - -
  | A B C D

Bot's move:
4 |   W B  
3 | B     W
2 | W B W  
1 |     B  
- + - - - -
  | A B C D

Player.BOT won!!!
